In [ ]:
````xml
<VSCode.Cell language="markdown">
# Quick Start Guide - Neural Execution Risk Predictor

This notebook provides a streamlined workflow to get started quickly.

**Steps:**
1. Install dependencies
2. Generate data
3. Train model
4. Test predictions
5. Start API

Let's begin! 🚀
</VSCode.Cell>

<VSCode.Cell language="code">
# Step 1: Check if running in the correct directory
import os
import sys

project_root = r"D:\JAVA\CODE\PYTHON\ML\DL\Neural Execution Risk Predictor"
os.chdir(project_root)

print(f"✓ Working directory: {os.getcwd()}")
print(f"\nProject structure:")
for item in os.listdir():
    print(f"  - {item}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 2: Install Dependencies

Run this cell to install all required packages.
</VSCode.Cell>

<VSCode.Cell language="code">
# Install requirements
!pip install -q -r requirements.txt

print("✓ All dependencies installed!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 3: Generate Dataset

We'll generate data from two sources:
1. BPI Challenge 2012 XES file (if available)
2. Synthetic execution plans
</VSCode.Cell>

<VSCode.Cell language="code">
# Check if XES file exists
xes_file = "new_BPI_Challenge_2012.xes"
xes_exists = os.path.exists(xes_file)

if xes_exists:
    print(f"✓ Found XES file: {xes_file}")
    print("  Will extract features from BPI Challenge data")
else:
    print(f"⚠ XES file not found: {xes_file}")
    print("  Will use synthetic data only")
</VSCode.Cell>

<VSCode.Cell language="code">
# Generate BPI features (if XES file exists)
if xes_exists:
    print("Extracting BPI features...")
    %run scripts/extract_bpi_features.py
    print("\n✓ BPI features extracted!")
else:
    print("Skipping BPI extraction (file not found)")
</VSCode.Cell>

<VSCode.Cell language="code">
# Generate synthetic plans
print("Generating synthetic execution plans...")
%run scripts/generate_synthetic_plans.py
print("\n✓ Synthetic plans generated!")
</VSCode.Cell>

<VSCode.Cell language="code">
# Combine datasets
import pandas as pd

data_dir = "data"
synthetic_path = os.path.join(data_dir, "synthetic_plans.csv")
combined_path = os.path.join(data_dir, "execution_risk_dataset.csv")

# Load synthetic data
df_synthetic = pd.read_csv(synthetic_path)
df_synthetic = df_synthetic.drop(columns=['plan_id'])

# If BPI data exists, combine; otherwise use synthetic only
bpi_path = os.path.join(data_dir, "bpi_features.csv")
if os.path.exists(bpi_path):
    print("Combining BPI + Synthetic data...")
    df_bpi = pd.read_csv(bpi_path)
    df_bpi = df_bpi.drop(columns=['case_id'])
    df_combined = pd.concat([df_bpi, df_synthetic], ignore_index=True)
else:
    print("Using synthetic data only...")
    df_combined = df_synthetic

# Shuffle and save
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)
df_combined.to_csv(combined_path, index=False)

print(f"\n✓ Combined dataset saved: {combined_path}")
print(f"  Total samples: {len(df_combined)}")
print(f"\nRisk distribution:")
print(df_combined['failure_label'].value_counts().sort_index())
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 4: Train Model

Now we'll train the neural network!
</VSCode.Cell>

<VSCode.Cell language="code">
# Train the model
print("Starting model training...\n")
%run model/train.py

print("\n✓ Model training complete!")
print("  Model saved to: model/risk_model.h5")
print("  Scaler saved to: model/scaler.joblib")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 5: Test Predictions

Let's test the trained model with sample inputs.
</VSCode.Cell>

<VSCode.Cell language="code">
import numpy as np
import joblib
from tensorflow import keras

# Load model and scaler
model = keras.models.load_model("model/risk_model.h5")
scaler = joblib.load("model/scaler.joblib")

print("✓ Model and scaler loaded!\n")

# Test samples
test_samples = [
    {
        "name": "🟢 Low Risk - Simple Execution",
        "features": [3, 2, 2, 0, 1500, 1, 0, 1, 60]
    },
    {
        "name": "🟡 Medium Risk - Moderate Complexity",
        "features": [8, 4, 4, 1, 6000, 3, 2, 2, 180]
    },
    {
        "name": "🔴 High Risk - Complex Execution",
        "features": [18, 8, 7, 1, 15000, 6, 12, 4, 500]
    }
]

risk_labels = ["LOW", "MEDIUM", "HIGH"]

print("="*60)
print("SAMPLE PREDICTIONS")
print("="*60)

for sample in test_samples:
    # Prepare input
    features = np.array([sample["features"]])
    features_scaled = scaler.transform(features)
    
    # Predict
    probs = model.predict(features_scaled, verbose=0)[0]
    predicted_class = np.argmax(probs)
    confidence = probs[predicted_class]
    
    print(f"\n{sample['name']}")
    print(f"  Prediction: {risk_labels[predicted_class]}")
    print(f"  Confidence: {confidence:.2%}")
    print(f"  Probabilities: LOW={probs[0]:.3f}, MED={probs[1]:.3f}, HIGH={probs[2]:.3f}")

print("\n" + "="*60)
print("✓ All predictions successful!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 6: Start API (Optional)

To start the FastAPI service, run this in a separate terminal:

```bash
cd "D:\JAVA\CODE\PYTHON\ML\DL\Neural Execution Risk Predictor"
uvicorn api.main:app --reload
```

Then access:
- **API Docs**: http://localhost:8000/docs
- **Health Check**: http://localhost:8000/health
</VSCode.Cell>

<VSCode.Cell language="code">
# Test API (if running)
import requests

try:
    response = requests.get("http://localhost:8000/health", timeout=2)
    if response.status_code == 200:
        print("✓ API is running!")
        print(f"  Response: {response.json()}")
        print("\n  Access docs at: http://localhost:8000/docs")
    else:
        print("⚠ API returned error")
except:
    print("ℹ API is not running")
    print("\n  Start it with:")
    print("  uvicorn api.main:app --reload")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🎉 Quick Start Complete!

You've successfully:
- ✅ Generated training data
- ✅ Trained the neural network
- ✅ Tested predictions
- ✅ (Optional) Started the API

### Next Steps:

1. **Explore the full notebook**: `neural_execution_risk_predictor.ipynb`
   - Detailed EDA and visualizations
   - Feature importance analysis
   - Error analysis

2. **Check evaluation metrics**: Run `model/evaluate.py`

3. **Test the API**: Run `test_api.py`

4. **Deploy with Docker**:
   ```bash
   docker build -t neural-risk-predictor .
   docker run -p 8000:8000 neural-risk-predictor
   ```

5. **Read the README**: Complete documentation and architecture details

---

**Have fun building safer agent systems!** 🚀
</VSCode.Cell>
````